In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
#week_day_Jul_list = [2, 9, 16, 23, 30]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2598)

In [15]:
x[:,:2]

matrix([[  4776.81570248,   5048.12748064],
        [  5123.00502512,   5087.17711624],
        [  7186.09961576,   6235.34264468],
        [  4648.46584343,   5404.64744614],
        [  5932.65271731,   6545.53094319],
        [  5670.57150601,   3859.14240892],
        [  5937.48055559,   8764.49742736],
        [  4576.11026836,   5850.08771821],
        [  7348.3718596 ,   8241.18128918],
        [  7343.34427156,   7840.87017112],
        [ 11746.99540223,  11692.411206  ],
        [  9560.87198556,  11690.91364377],
        [  6306.45897434,   6730.19131819],
        [  6986.25030835,   5102.35653027],
        [  2692.5091086 ,   5321.00483096],
        [  4146.81417148,   2159.83013414],
        [  4201.10753243,   5410.28662518],
        [  3672.00987576,   3675.50080834],
        [ 13794.5170043 ,  15186.65225442],
        [ 10657.92411621,   7537.76860231],
        [ 18423.89350562,  16733.09665144],
        [ 10402.30490222,  13513.18536584],
        [  3173.98976307,   5497

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [6e-01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -5.66637254e+06  0.00000000e+00  5.40e+03 3.03e+02  8.64e+05     0s
   1   1.11594108e+06 -3.32091382e+06  4.66e+02 7.26e+01  1.57e+05     0s
   2   9.53557118e+04 -1.13526124e+06  4.66e-04 5.68e-14  2.20e+04     0s
   3  -2.48368487e+05 -4.00473784e+05  4.66e-10 2.84e-14  2.7

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.0299629541323759e-08,
 9418.640994246269,
 1.1365627546172768e-08,
 8.06369134837874e-06,
 3200.0820440270422,
 1.3519494972751388e-05,
 2.7377071083174874e-06,
 0.012219899842635981,
 1.0773868037537659e-07,
 5226.363010322432,
 1.0631816728639273e-07,
 1.0773868040108647e-07,
 1.0688688435515663e-07,
 1.0749981581471694e-07,
 4901.779587837813,
 1938.220287386406,
 8.063690987153272e-06,
 8.063691218551228e-06,
 4302.233930168045,
 14927.478690799118,
 1.3516620966346862e-05,
 3238.8270842518064,
 2693.4047200392856,
 437.19478284581277,
 1551.9926443533373,
 1162.837197518109,
 566.2767473338288,
 4460.924887944706,
 0.4693976586809754,
 1.8892139751646937,
 1078.1652564595424,
 2915.7040208511767,
 4498.134433501735,
 603.6308159368832,
 581.7477869985551,
 3525.481100490172,
 1616.6200140379513,
 0.00023595804760056702,
 1375.929972749424,
 3310.9453852984734,
 6246.669054708841,
 1501.1021301223664,
 9.733907644772434e-05,
 9.73390976635566e-05,
 12541.103547034634,
 5.5486488